生成新版synthetic archetype

每批30/5故事生成一个版本的synthetic archetype。一共还是5个版本，只不过每个版本都来自不同的30/5个故事。


In [1]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  # 导入tqdm用于显示进度条

In [2]:
from openai import OpenAI
# 设置 API 密钥和组织
api_key = "sk-WO49kazJ9a1PDGbE2lPRhJ9vDG8hak5_mbD60YhIB8T3BlbkFJHbhSLNYAPwzhLHjS72qabHWqGwtBmAvgRpRwN_iYMA"
client = OpenAI(api_key=api_key)

# 设置组织信息
client.organization = "org-34qJHjOz4UGZYVqv2zlohVMe"

1. 拼接获得故事风格摘要（30）

In [3]:
# 读取并拼接故事风格摘要
def load_style_summary(csv_path):
    df = pd.read_csv(csv_path)
    # 将“故事风格摘要”列的所有内容拼接成一个字符串
    style_summary = " ".join(df["故事风格摘要"].dropna().astype(str).tolist())
    return style_summary

2. 获取reference story（5）

3. 调用生成synthetic archetype的prompt

In [4]:
def generate_story_with_json_conversion(style_summary, theme, explanation, reference_stories):
    # 构造参考故事部分
    references = "\n\n".join([f"参考故事{i+1}：{story}" for i, story in enumerate(reference_stories)])
    
    # 构造完整的提示语，要求直接生成符合格式的 JSON 输出
    prompt = f"""
    你是一位擅长撰写中国民间故事的作家。请根据以下要求生成一个中国民间故事：
    1. 具体要求：{explanation}
    2. 写作时请参考以下信息：
    - 故事风格特点摘要：{style_summary}
    - 五个参考故事：
    {references}

    要求：
    1. 故事必须紧紧围绕具体要求，不要过度解读或发散。
    2. 写作风格和时代背景请参考上述信息，字数控制在约1000字左右。
    3. 故事内容应完整，不得包含额外的说明或分析。
    4. 另请单独提供一段”justification”内容，在其中简要解释该故事如何符合具体要求。此说明应作为单独字段输出，不应出现在故事正文中。

    请严格按照以下 JSON 格式输出，不包含额外文字或提示，并确保格式有效：
    {{
    “theme”: “{theme}”,
    “story”: “这里是生成的故事内容”,
    “justification”: “这里是对故事与主题相关性说明的描述”
    }}
    """

    try:
        # 直接使用 o3-mini 模型生成符合要求的 JSON 格式结果
        response = client.chat.completions.create(
            model="o3-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        # 直接返回生成的 JSON 字符串，不再额外进行转换或解析
        result = response.choices[0].message.content.strip()

        # 确保返回有效 JSON
        try:
            result_json = json.loads(result)
            return json.dumps(result_json, ensure_ascii=False, indent=4)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
            return json.dumps({"error": "Error generating JSON format"}, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"Error generating story for {theme}: {e}")
        return json.dumps({"error": "Error generating story"}, ensure_ascii=False, indent=4)


In [5]:
# 定义处理每一行的函数
def process_row(row, style_summary, reference_stories):
    theme = row['Theme']  # 获取主题名称

    results = []
    # 遍历从第二列到最后一列，逐一传递给generate_story函数
    for column in ['Generic', 'Specific_Positive','Specific_PositiveA', 'Specific_PositiveB', 'Specific_Negative', 'Specific']:
        explanation = row[column]
        if pd.notna(explanation):  # 如果该列有内容
            story_json = generate_story_with_json_conversion(style_summary, theme, explanation, reference_stories)
            print(f"Generated story JSON for theme '{theme}' and column '{column}': {story_json}")
            story_data = json.loads(story_json)
            
            # 组合数据并保存
            result = {
                'Theme': theme,
                'Explanation': explanation,
                'Explanation_Column': column,  # 保存当前列名
                'Story': story_data.get('story', ''),
                'Justification': story_data.get('justification', '')
            }
            results.append(result)
    
    return results

# 定义并行处理的主函数，使用tqdm显示进度条
def process_rows_parallel(df, style_summary, reference_stories):
    all_results = []
    
    # 使用ThreadPoolExecutor进行并行处理，并用tqdm显示进度条
    with ThreadPoolExecutor(max_workers=45) as executor:  # 设置最大并行工作线程数为45
        # 提交任务，并将任务与进度条结合
        futures = [executor.submit(process_row, row, style_summary, reference_stories) for index, row in df.iterrows()]
        
        # 使用tqdm显示进度
        for future in tqdm(futures, desc="Processing rows", total=len(futures)):
            result = future.result()
            all_results.extend(result)
    
    return all_results

In [6]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 假设你已经定义好 load_style_summary, generate_story_with_json_conversion,
# process_row, process_rows_parallel 等函数

# 定义 df 文件路径（同一个）
df_file_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/250218latest_prompts_for_synthetic_archetypes.xlsx"
df = pd.read_excel(df_file_path)
# print(df.head())

# 循环处理每个版本
for version in range(1, 6):
    # 构造 style_summary 文件路径
    style_summary_file = f"/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250219story_styles_summary/250219story_styles_summary_version4.csv"
    style_summary = load_style_summary(style_summary_file)
    # print(style_summary)
    # 构造 reference_stories 文件路径
    reference_stories_file = f"/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250219reference_story_in_prompt/250219reference_story_in_prompt_version3.xlsx"
    df_ref = pd.read_excel(reference_stories_file, skiprows=1)
    # print(df_ref.head())
    # 检查列名是否符合预期，假设列名依然是 'Tale Name' 和 'Tale Content'
    if 'Tale Name' in df_ref.columns and 'Tale Content' in df_ref.columns:
        sampled_stories = df_ref.to_dict(orient='records')
        # print(f"✅ Version {version}：参考故事数据加载成功")
    else:
        print(f"❌ Version {version}：CSV文件缺少预期的列('Tale Name', 'Tale Content')")
        continue
    
    reference_stories = [story["Tale Content"] for story in sampled_stories]
    # print(reference_stories)
    # 调用并行处理函数
    results = process_rows_parallel(df, style_summary, reference_stories)
    
    # 定义输出文件路径
    output_file_path = f"/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250228generated_synthetic_story_version{version}.xlsx"
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_file_path, index=False)
    
    print(f"版本 {version} 的故事和解释已成功保存到 {output_file_path}")

Processing rows:   0%|          | 0/26 [00:00<?, ?it/s]

Generated story JSON for theme '规则遵循（Rule-Following）' and column 'Generic': {
    "theme": "规则遵循（Rule-Following）",
    "story": "很久以前，在青山环抱的南阳村里，流传着一条自古以来不可违背的村规：每逢秋收时节，所有村民必须将收获的第一筐谷物送往庙会，以示对天命的感恩，任何人若私自截留，必遭不测。村里人世代以此为戒，既有敬畏之心，也有暗自调侃的智慧。\n\n村中有个名叫王诚的年轻人，聪明伶俐但又偶有小聪明。他跟随父辈耕耘年年遵规，但心内暗自打算，今年若能私藏一点，好去城里做买卖，赚点外快。村中老前辈刘道人看出了王诚的隐秘念头，曾语重心长地对他说：“规矩乃天理，若扰了这天命，日后连天公也不放过你！”王诚初听不以为意，暗想刘道人就是个迷信老头，心想自有办法解决。\n\n到了丰收的季节，村里一片忙碌景象。大伙儿按规矩把刚刚收割的谷物依序送至庙堂，伴随祭祀仪式，笑谈间颇觉仪式繁琐。唯独王诚心中盘算着偷偷留下一筐最上好的谷物。深夜，他趁众人熟睡，悄悄将一筐金黄谷粒推入自家谷仓中。正当他得意地轻声自笑时，忽传来门口敲门声，竟是刘道人一脸严肃地站在门前：“诚儿，那筐谷物似乎少了一点，莫非你敢私吞规矩的礼物？”\n\n王诚心中一慌，连连辩解道：“大师傅，这年头粮食丰足，留下一点算不得什么……”刘道人冷冷一笑：“村规如天，每一粒谷物都寓意着祖先传下的不变信仰。你若敢违背，天命自会显灵，莫怪日后连邻里笑话你！”\n\n王诚不以为意之际，村中果然出现了奇异现象。第二日早晨，他发现仓中那筐谷物竟分崩离析，残存仅余零散谷粒；而屋外的鸡犬也流露出异样的惊恐。邻居老赵看热闹说道：“这规矩才是真正的神明使者，私存之举必会招来不祥！”村里的笑谈与调侃中隐含着对规矩不可亵渎的敬畏。\n\n眼见自己谋利未果，王诚心生悔意，急忙到庙中求原谅。庙会正进行得热闹非凡，众人正在欢庆天恩，而老庙主持郑师傅正诵经祈福。王诚伏在膝前，断断续续地道歉，并将所有剩余谷粒一同供奉于神案前。就在他诚恳忏悔时，村中忽又传来消息：前日误留私货的地主也遭遇了连番霉运，生意败落、家中频频起火。众人议论纷纷，皆认为那是不尊重规矩的报应。\n\n此后，王诚重整信心，严格遵守村规，每年按时捐赠谷物。他也渐渐明白，规

Processing rows: 100%|██████████| 26/26 [01:36<00:00,  3.71s/it]


Generated story JSON for theme '信任（Trust）' and column 'Specific_Negative': {
    "theme": "信任（Trust）",
    "story": "很久很久以前，在北方一个偏僻的小村庄里，住着一个勤劳朴实的农夫，名叫陈大福。陈大福为人热心，平日里乐于助人，可他却有一个致命的弱点——太容易轻信别人。一天，陈大福在集市上碰到了一位衣衫褴褛、口齿伶俐的游方道人，自称叫老李。老李低声细语地说道：“大福兄弟，你可知我曾得灵药秘方，只需投入几两银子，便可让你家中生意兴隆，福泽绵长！”陈大福本就心软，加之眼前这老李言辞恳切，于是一边听一边点头，毫不犹豫地把积攒多年的家产交给了他，并许诺第二日再来取回“灵药”的所在之处。\n\n次日清早，陈大福便背起粗布包袱，按照老李之前的指点，前往村外那座传说中能迎来吉兆的荒山。路上，他遇到了邻居阿文。阿文见状急忙劝道：“大福哥，这种天上掉馅饼的事往往不可信，你可得多加小心。”陈大福大笑答道：“阿文，世上哪有白吃羹的事，老李的话我信得牢牢的！”说完，便执意上路。\n\n沿着崎岖的小路，陈大福走了好些时辰，天边逐渐暗了下来。忽然，前方出现了一队自称为“引泉人”的汉子，他们神色阴沉，拦住了归途的行人。领头的汉子冷冷说道：“原来是陈大福啊，听闻老李说你有缘得灵泉，现需再筹资物，才能真见那泉水奇景。”陈大福心中虽有一丝迟疑，但想到老李曾言的神奇之事，便仍抱侥幸心态，掏出大半家财。那“引泉人”见状，露出狞笑，毫不留情地又索取了部分杂物与值钱物品，说是用于供奉山神。\n\n待一切手续办妥后，他们带着陈大福继续深入山谷。然而，越走山路越是险恶，四周浓云密布，野兽嘶吼声不绝于耳。陈大福渐生后悔之意，却已无力回天。正当他苦苦哀求时，那些人早已人间蒸发，只留下满怀欺骗与绝望的他独自面对荒野的寒风凛冽。\n\n回到村中后，陈大福发现家中早已门徒空空。邻里纷纷议论，有人讥笑他“轻信便捞尽，教人落得家破人亡”，更有老者叹道：“信任如同双刃剑，你若不多加防备，便会被狡诈之人利用！”从此以后，陈大福日子愈发凄凉，他再也没有敢全心相信任何人，面对即便是真心相待的邻居也总提心吊胆。岁月匆匆流过，他回忆起往昔时光，满怀遗憾地喃喃自语：“我若能谨慎些，何至于今日穷困潦倒？须知独立思考和怀疑精神，才是

Processing rows:   0%|          | 0/26 [00:00<?, ?it/s]

...

Generated story JSON for theme '信任（Trust）' and column 'Specific_Negative': {
    "theme": "信任（Trust）",
    "story": "在古老的南山脚下，有个叫青岩的小村庄。村里有个勤劳善良的青年，名字叫赵诚，他从小家境清贫，一直相信世间人人为善。某天，他路过镇上，遇见了一位看似和蔼的中年人柳伯。柳伯推销着自己的新发明——一种据说可以催熟果实、提高收成的神奇“金禾水”。柳伯言辞恳切地对赵诚说：“小伙子，咱这金禾水乃是我在远方采来的神水，只要用它浇灌庄稼，收成定翻番。你若信我一回，当真日后丰收在望。”\n\n赵诚本性单纯，便打心眼儿相信了柳伯的说辞。他对柳伯道：“伯，你说得这么真切，我怎么敢不信？我这点积蓄虽不多，但愿赌服输，试试这金禾水。”柳伯见状，连连点头，随即写下了一张纸条，声称只需拿出三两银子作为试用费用，便可得神奇金禾水。赵诚虽存疑虑，却被柳伯的话语感动，便将所有积蓄换成三两银子，交给柳伯。柳伯收过钱后，嘱咐道：“这金禾水取自高山云泉，平日里用来灌溉，必能使庄稼生长得更快。记住，凡事只需按时用水，自然有回报。”说完便消失在人群中。\n\n赵诚兴冲冲地回到村里，心中满是对未来丰收的憧憬。他按照柳伯的吩咐，将金禾水均匀洒在自家田里。村中老农听了也是半信半疑，纷纷劝他：“人不可太轻信陌生人，天下哪有这么好的事？”赵诚笑道：“我看今日种下的定是好收成，你们不妨观望几天再说。”\n\n然而，日子一天天过去，他的田地却依然枯黄。一番凌乱后，赵诚焦急地找上了村中一位经验老道的农业师傅，师傅看了看田地，叹气道：“这所谓金禾水，不过是一碗普通水，加上些许劣质药剂，实在对庄稼无益，反而伤了根基。”赵诚一听，心中顿起凄凉，意识到自己被欺骗了。他连忙去镇上打听柳伯的下落，竟发现柳伯早已离镇而去，踪迹全无。\n\n失魂落魄的赵诚只得返回田间，望着一片萧条，喃喃自语：“我一心信任别人，竟让自己陷入这般窘境。”就在人群议论纷纷之际，赵诚的表兄赵德见状，走上前来安慰道：“诚弟，这世道险恶，轻信他人往往容易受损。你如今虽遭此劫，但也得记住，今后要多加提防，独立作主、谨慎为先。”赵诚听了，满腔懊悔，泪眼婆娑地说道：“我过于天真，甘愿把希望寄托在别人虚伪的承诺上，现在，方知信任有时也会成陷阱。”\n

Processing rows:   8%|▊         | 2/26 [00:58<10:14, 25.60s/it]

Generated story JSON for theme '耐心（Patience）' and column 'Specific_Negative': {
    "theme": "耐心（Patience）",
    "story": "很久以前，在东山脚下的一个小村庄里，住着一个叫何德昌的年轻木匠。他素有“沉稳耐心”之称，凡事总是一副慢条斯理的模样。村中人都说：‘德昌这孩子，干什么事都要慢慢来，好比等花开似的。’\n\n一天，远道而来的商贩李仁带着一则消息踏入村庄：“今天市上将举行奇货大拍卖，能卖出高价的，便可转身致富！”邻里的握手言欢，都迫不及待奔向市集准备展示各自的手艺和货物。只有何德昌站在自家门前，望着远去的人群，轻轻摇头道：“耐心是美德，好货总会有好价，何必匆忙？”\n\n他的好友赵兴明见状，急忙劝道：“德昌，市上机会难得，你再这样等下去，可就错过了啊！”何德昌笑呵呵地回答：“兴明，你看这木头，要自然晾干，时间不可催促。我的手艺是多年磨一剑，更需等待时机成熟。”\n\n市集那边热闹非凡，各家摊位前人头攒动。李仁当日正是为了寻找那些精雕细刻、器形独特的手工艺品，预谋大赚一笔。他不厌其烦地在各摊位之间穿梭，生怕错失每一件宝物。不少摊主看到他满面喜色，也纷纷降价吆喝，好让自己的货物早早走俏。\n\n而何德昌本想拿出自家珍藏的木雕屏风，那是他苦心孤诣的作品，历经数月精雕细琢，纹理流畅，雕工精致。但他心中一直盘算着，要等到那个所谓“完美时机”，再敢出手。于是，他选择把它珍藏在自家土窖中，反复推敲：“若现在卖，价格怎能尽人意？先等待一等，定能高价成交。”\n\n时间一天天过去，市集上的行情却转瞬即逝。那次大拍卖不仅炒出了高价，一位眼光独到的富商更当场斥巨资收购了许多艺术珍品。然而，当何德昌正准备出门赶往市集时，他的邻居兴明急匆匆地跑来：“德昌，快快快！富商已走，剩下的都是些败絮。但你的那件正品还在家中呢！”\n\n何德昌满脸不甘，回忆内心反复权衡，便固执地说：“一切皆因时间不成熟，耐心方能换来真正的价值。若急于出手，岂非与粗制滥造为伍？”兴明叹道：“德昌，你这过分等待，错失的可是良机啊！有时候，机遇就在眼前，偏偏被一味等待的心耽误了。”\n\n当晚，何德昌辗转难眠，窗外月光如练，一轮圆月洒下银辉。他心中窃自叹息：‘走过的时光已不复返，那最佳的时机也消失得无影无

Processing rows: 100%|██████████| 26/26 [01:09<00:00,  2.66s/it]

Generated story JSON for theme '慈善（Charity）' and column 'Specific_Negative': {
    "theme": "慈善（Charity）",
    "story": "从前，在北方一个偏远的小村庄里，人们世代以耕读传家，互助为乐。村中有位叫赵仁的好汉，生得宽厚仁慈，为人乐于助人。每逢村里有贫困之家，他总是慷慨解囊，送上口粮和银两。久而久之，赵仁的善心名扬四邻，渐渐地，村里不少人便把他的资助当成了日常，不再勤奋自强。\n\n村东角住着个叫阿根的懒汉，自幼家境贫寒，靠着赵仁施舍过日。起初，阿根总心存感激，一边吃着赵仁赠送的饭食，一边发誓改变命运；但时间一长，他便沉溺于依赖，不肯动手谋生。有时，他还结伴邻居高庆、李旺等人，边议论赵仁的仁慈，边幻想着能混吃等死。有人劝说：“阿根，你再这样下去，不思进取，将来日子更苦！”阿根只是摇头笑道：“赵仁老弟这么大方，日后必定还有扶持，何必自己费劲苦干？”\n\n渐渐地，整个村庄似乎都笼罩上了一种无为的气氛。原本热心经营小生意的吴老板也因靠慈善商品打头阵而失去了竞争力，生意日渐凋零。市场上由赵仁补贴而来的农产品，价格虚高，致使本来兢兢业业耕作的农户也开始倚重摊派，斤斤计较捞取微利。村内逐步分化为两派：一派依赖于赵仁无偿援助，失去自立能力；另一派则努力自强，却因失去了活跃市场而倍感压力。\n\n这一切变化，村中年长的古伯看在眼里，急在心头。一天午后，他邀来赵仁与几位村中青年，在古槐树下商议对策。古伯语重心长地说：“仁子，你那一片赤诚之心本无瑕疵，可如今施善之法不当，反成累人。助人固然美德，但若让人赖着不求上进，反而扭曲了市场秩序，连勤劳人家也受牵连。”\n\n赵仁低头沉思，心中涌起一阵悔意。他问道：“古伯你说得对，可如何才能既施以援手，又不令受助者失去自立之志？”此时，一位衣着简朴的陌生人从树后走出，自称风灵，他道：“真正的慈善，不在于无条件的资助，而在于传授自立之道。你们可以设立技艺讲习、创业指导，帮助穷人学本领，而非单纯给予温饱。”\n\n受此启迪，赵仁召集村民大会，提出一项新的善举：将全村部分资源投入建立一所‘自强学堂’，每逢贫困户前来求助时，不再直接发放救济，而是先安排技艺培训、劳动指导，待其掌握生计技能后，再给予适度扶持。一时间，原先习惯赖着恩惠的阿根、及